# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/data/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

c:\Users\OmegaCel\Documents\GitHub\Data-Modeling-with-Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""create keyspace if not exists sparkify 
                   with replication = {'class': 'SimpleStrategy' , 'replication_factor': 1 }
                """)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we will create tables based on the below queries. With Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## QUERY 1
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [8]:
session.execute("""
    CREATE TABLE IF NOT EXISTS session_log
    (sessionId int
    , itemInSession int
    , artist text
    , song text
    , song_length float,
    PRIMARY KEY(sessionId, itemInSession))
    """)              

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_log (sessionId, itemInSession\
            , artist, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
#Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, song_length from session_log\
     WHERE sessionId=338 AND itemInSession=4"
rows = session.execute(query)
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


## QUERY 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)  for userid = 10, sessionid = 182

In [ ]:
session.execute(""" 
CREATE TABLE IF NOT EXISTS event_log
(userId int
, sessionId int
, itemInSession int
, firstName text
, lastName text
, artist text
, song text
, PRIMARY KEY ((userId, sessionId), itemInSession))
        """)

                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO event_log (userId, sessionId, itemInSession, firstName, lastName, artist, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT artist, song, firstName, lastName from event_log WHERE userId=10 AND sessionId=182"
rows = session.execute(query)
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## QUERY 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
session.execute(""" 
CREATE TABLE IF NOT EXISTS song_log
(song text 
, userId int
, firstName text
, lastName text,
, PRIMARY KEY (song, userId))
        """)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_log (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
#Query 3 Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT firstName, lastName from song_log WHERE song = 'All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

session.execute("DROP TABLE song_log")
session.execute("DROP TABLE event_log")
session.execute("DROP TABLE session_log")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()